In [ ]:
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using Revise, Genie, DCAUtils, DelimitedFiles, PyPlot
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using Revise, Genie, DCAUtils, DelimitedFiles, PyPlot, JLD2, CSV, DataFrames

@load "../data_Genie/pars_dbd.jld2"

fam_file = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m"
eq_file = "../Gen.jl/data/chains/equil_det_bal_dbd_silico_chain_num_1_T_1.0.fasta"  #"../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m"
fam_name = "DBD"
out_folder = "../equilibrium_$(fam_name)_dms_langevin/"
clean_nat_msa = remove_duplicate_sequences(read_fasta_alignment(fam_file, 0.9))[1];
eq_msa  = remove_duplicate_sequences(read_fasta_alignment(eq_file, 0.9))[1];
q = 21; L, M = size(clean_nat_msa);
out_folder = "../equilibrium_$(fam_name)_dms_langevin/"


@time for idx in 1:M
    ps = zeros(q, L)
    for i in 1:L
        ps[:,i] .= Genie.single_site_prob_cond(i, eq_msa[:,idx], h_dbd, J_dbd, L, q = q);
    end
    
    filepath = joinpath(out_folder,"$(fam_name)_wt$(idx).txt")

    open(filepath, "w") do io
        # Write the header for the output file, using tabs as delimiters
        println(io, "i\tAA\tp")
        for i in 1:L
            for a in 1:q
                println(io, "$i\t$a\t$(ps[a,i])")
            end
        end
    end
    
end


n_dim = 5
pc_nat, PC, expl_var = perform_pca(clean_nat_msa, n_dim = n_dim);
q = 21; L, M = size(clean_nat_msa); Lq =  L*q;
    

mean_forces = zeros(n_dim, M)
@time for idx in 1:M
    filepath = joinpath(out_folder,"$(fam_name)_wt$(idx).txt")
    data = CSV.read(filepath, DataFrame, delim='\t')
   
    msa = zeros(Int8, L, Lq)
    for n in 1:Lq
        msa[:,n] = copy(eq_msa[:,idx])
        msa[data.i[n],n] = data.AA[n]
    end
    
    pc_sil = Genie.get_projection(PC, msa);
    
    mean_forces[:,idx] .= [mean(pc_sil[:,n], AnalyticWeights(data.p)) for n in 1:n_dim] .- pc_nat[idx,:]
end



ens = energy(eq_msa, h_dbd, J_dbd);

mean_energies = zeros(M)

probs = [];
delta_energies = [];
m = 1

@time for idx in 1:M
    #println(idx)
    filepath = joinpath(out_folder,"$(fam_name)_wt$(idx).txt")
    data = CSV.read(filepath, DataFrame, delim='\t')
   
    msa = zeros(Int8, L, Lq)
    for n in 1:Lq
        msa[:,n] = copy(eq_msa[:,idx])
        msa[data.i[n],n] = data.AA[n]
    end

    en_msa = energy(msa, h_dbd, J_dbd)
    d_energies = en_msa .- ens[idx]

   
    push!(probs,data.p...)
    push!(delta_energies, d_energies...)
    
    mean_energies[idx] = mean(en_msa .- ens[idx], AnalyticWeights(data.p)) 
end


pc_eq = Genie.get_projection(PC, eq_msa);
new_x = pc_nat  +  mean_forces'
new_dE = [ens[argmin(sum(abs2, pc_nat .- new_x[i,:]', dims = 2))[1]] - ens[i] for i in 1:M];

close("all")
plt.hist(mean_energies, histtype = "step", linewidth = 3.0,  label = "<dE> through gibbs")
plt.hist(new_dE, histtype = "step", linewidth = 3.0,  label = "<dE> through closest nat in PCA")
plt.xlabel("dE (DCA)")
plt.legend()
savefig("../mean_dE_from_eq.png")



using PyPlot # Sostituisce 'using Plots'
using StatsBase # Necessario per AnalyticWeights
using Random
using Printf



# Imposta il seed per la riproducibilità, se desiderato
Random.seed!(42) 

M_prime = 500 # Numero di sequenze da selezionare casualmente per le FRECCE (Vettori di Forza)

# --- Preparazione Dati Totali per lo Scatter Plot ---

total_sequences = size(eq_msa, 2)

ens_nat = energy(clean_nat_msa, h_dbd, J_dbd)
# Estrai TUTTI i dati per lo Scatter Plot di sfondo
X_all = pc_nat[:, 1]  # PC1 (X-coordinate)
Y_all = pc_nat[:, 2]  # PC2 (Y-coordinate)
C_all = ens_nat       # Energy (Colore)

# --- Preparazione Dati per i Vettori di Forza (Solo M_prime Casuali) ---

n_small = 1000; 
idxs = sample(1:M, n_small, replace=false)
# Estrai i dati per le frecce solo per gli indici selezionati casualmente
X_arrows = pc_eq[idxs, 1]  # PC1 per le frecce
Y_arrows = pc_eq[idxs, 2]  # PC2 per le frecce
U = mean_forces[1, idxs] # Componente PC1 della Forza Media
V = mean_forces[2, idxs] # Componente PC2 della Forza Media

# Normalizzazione dei vettori di forza per una migliore visualizzazione
# Usa il range dei dati TOTALI (X_all, Y_all) per una normalizzazione coerente.
max_pc_range = max(maximum(X_all) - minimum(X_all), maximum(Y_all) - minimum(Y_all))
max_force_mag = maximum(sqrt.(U.^2 + V.^2))

# Ridimensiona per avere una lunghezza massima pari al 5% del range totale dell'asse
scale_factor = 0.25 * max_pc_range / max_force_mag
U_scaled = U * scale_factor
V_scaled = V * scale_factor

# --- Plotting con PyPlot ---

figure(figsize=(8, 8))
ax = gca()

# 1. Grafico a dispersione (Scatter Plot) - PLOTTA TUTTI I PUNTI
scatter(
    X_all, Y_all,
    c = C_all,           # Colora i punti in base a C (Energy)
    cmap = "viridis",
    s = 20,          # Dimensione ridotta per non oscurare troppo lo sfondo
    edgecolors = "none", # Nessun bordo per i punti di sfondo
    alpha = 0.6
)

# Aggiunge la Color Bar
colorbar(label="DCA Energy (ens)")

# 2. Aggiungi i vettori di Mean Force (Quiver Plot) - SOLO PER M_prime PUNTI CASUALI
quiver(
    X_arrows, Y_arrows,
    U_scaled, V_scaled,
    color = "black",
    scale = 1,
    angles = "xy",
    scale_units = "xy",
    width = 0.005, # Spessore della linea del vettore
    headwidth = 5,
    headlength = 7,
    alpha = 0.9
)

# Impostazioni finali del grafico
title("DBD")
xlabel("Principal Component 1")
ylabel("Principal Component 2")
gca().set_aspect("equal") # Imposta l'aspetto 1:1

savefig("../eq_ciao_allpoints_Mprimerandomarrows.png")

using PyPlot 
using StatsBase 
using Random
using Printf

# --- General Setup ---

Random.seed!(42) 

M_prime = n_small # Numero di sequenze per le frecce
n_dim = 5      # PCA dimensions (PC1...PC5)
total_sequences = size(pc_nat, 1)

# --- Define the 10 PC pairs and their subplot positions ---
# La posizione è l'indice 1-based in una griglia 4x4.
# (PC_x, PC_y, posizione_1-based, numero_di_colonne_da_saltare)
# Nota: La griglia 4x4 ha 16 celle. L'indice va da 1 a 16.
pc_plot_map = [
    # Riga 1 (1 Plot, centra nell'indice 1 o 2 e usa una colonna)
    # Visto che non possiamo "centrare" facilmente, usiamo la prima cella della riga
    (1, 2, 1, 3), # PC1 vs PC2 (Indice 1, salta 3 celle)

    # Riga 2 (2 Plots)
    (1, 3, 5, 0), # PC1 vs PC3 (Indice 5, salta 1 cella)
    (2, 3, 6, 2), # PC2 vs PC3 (Indice 7, salta 1 cella)
    
    # Riga 3 (3 Plots)
    (1, 4, 9, 0), # PC1 vs PC4 (Indice 9, salta 0 celle)
    (2, 4, 10, 0), # PC2 vs PC4 (Indice 10, salta 0 celle)
    (3, 4, 11, 1), # PC3 vs PC4 (Indice 11, salta 1 cella) 
    
    # Riga 4 (4 Plots)
    (1, 5, 13, 0), # PC1 vs PC5 (Indice 13)
    (2, 5, 14, 0), # PC2 vs PC5 (Indice 14)
    (3, 5, 15, 0), # PC3 vs PC5 (Indice 15)
    (4, 5, 16, 0)  # PC4 vs PC5 (Indice 16)
]
# Nota: La mappatura sopra cerca di posizionare i grafici in modo che sembrino usare 1, 2, 3, 4 colonne,
# ma in realtà manipola solo il contatore degli indici per saltare le celle.

# --- Prepare Data (Assuming pc_nat, ens, mean_forces are available) ---

C_all = ens_nat[:] 

random_indices = idxs

# --- Force Normalization (5D) ---

max_pc_range = maximum([maximum(pc_nat[:, i]) - minimum(pc_nat[:, i]) for i in 1:n_dim])
force_components = [mean_forces[i, random_indices] for i in 1:n_dim]
max_force_mag = maximum(sqrt.(sum(c.^2 for c in force_components)))

scale_factor = max_pc_range / max_force_mag
mean_forces_scaled = 0.25 * mean_forces * scale_factor

# --- PLOTTING: Manipolazione Manuale degli Indici ---

fig = figure(figsize=(20, 20))

# Contatore manuale per l'indice di posizione
manual_idx = 1
# Griglia base: 4 righe, 4 colonne
n_rows = 4
n_cols = 4

for (i, (pc_x_idx, pc_y_idx, start_pos, skip_cols)) in enumerate(pc_plot_map)
    
    # 1. Calcolo l'indice di subplot in base alla posizione iniziale (start_pos)
    # L'indice posizionale per subplot(R, C, I) è 1-based
    subplot_idx = start_pos
    
    # Crea il subplot. Usiamo un indice statico per simulare la griglia 4x4
    # ATTENZIONE: Se un subplot viene disegnato su una cella, la cella non è più libera.
    # L'unica cosa che possiamo fare senza GridSpec è non disegnare sulla cella.
    ax = subplot(n_rows, n_cols, subplot_idx)
    plt.sca(ax) 
    
    # 2. Extract Data for current PC pair
    X_all = pc_nat[:, pc_x_idx]
    Y_all = pc_nat[:, pc_y_idx]
    X_arrows = pc_eq[random_indices, pc_x_idx]
    Y_arrows = pc_eq[random_indices, pc_y_idx]
    U_sub = mean_forces_scaled[pc_x_idx, random_indices]
    V_sub = mean_forces_scaled[pc_y_idx, random_indices]
    
    # --- SCATTER PLOT (All Points) ---
    scatter(
        X_all, Y_all,
        c = C_all,
        cmap = "viridis",
        s = 8, 
        edgecolors = "none",
        alpha = 0.6
    )

    # --- QUIVER PLOT (M_prime Random Arrows) ---
    quiver(
        X_arrows, Y_arrows,
        U_sub, V_sub,
        color = "black",
        scale = 1,
        angles = "xy",
        scale_units = "xy",
        width = 0.003, 
        headwidth = 5,
        headlength = 7,
        alpha = 0.9
    )

    # 3. Set Axis Properties 
    #ax.set_title("PC$(pc_x_idx) vs. PC$(pc_y_idx)", fontsize=12)
    ax.set_xlabel("PC $(pc_x_idx)")
    ax.set_ylabel("PC $(pc_y_idx)")
    ax.set_aspect("equal") 
end

# --- Final Figure Adjustments and Colorbar ---

# Non c'è bisogno di eliminare subplot vuoti se non sono mai stati disegnati,
# ma dobbiamo nascondere le celle vuote (es. 2, 4, 8, 12, 16) per simulare lo spazio
# La cella vuota della riga 3 è la posizione 12 (3*4 = 12, in realtà 12 in Julia è la 12esima cella)
# Le celle non disegnate sono (2, 4), (3, 4), (4, 4) nella griglia 4x4.
# (Posizioni 8, 12, 16 - ma 16 è usata, quindi solo 8 e 12)

# Cellule da nascondere (solo per pulizia, se non sono usate)
# Ad esempio, la cella 12 (4a cella della riga 3)
try
    ax_hide = subplot(n_rows, n_cols, 12)
    ax_hide.set_visible(false)
catch e
    # Non fare nulla se la cella non esiste o è già usata
end


# 2. Add a common Color Bar
fig.subplots_adjust(right=0.9, top=0.95) 
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7]) 

sm = PyPlot.cm.ScalarMappable(cmap="viridis") 
sm.set_array(C_all)

colorbar(sm, cax=cbar_ax, label="DCA Energy (ens)")

fig.suptitle("DBD: Multi-Dimensional PCA con Struttura a Gradini (1, 2, 3, 4)", fontsize=18)
savefig("../eq_ciao_manual_grid_pca.png")



In [ ]:
      
   
function find_mutations(matrix::AbstractMatrix{<:Integer}, filepath::String, verbose = false)
    # 1. Input Validation and Setup
    num_sites, num_steps = size(matrix)
    if num_steps < 2
        error("Matrix must have at least two columns (reference and at least one subsequent sequence).")
    end

    # We are comparing N-1 steps: C2 vs C1, C3 vs C2, ..., Cn vs C(n-1)
    if verbose == true
        @info "Starting step-wise mutation analysis on $(num_steps - 1) steps."
    end
    
    # 2. File Writing
    # Use 'do io' block for safe file handling (ensures file is closed)
    open(filepath, "w") do io
        # Write the header for the output file, using tabs as delimiters
        println(io, "Site\tRef_AA\tNew_AA\tMutation_Step")

        # 3. Iterate over the steps (columns 2 through end)
        for col_index in 2:num_steps
            # The reference for *this step* is the previous column (C_{i-1})
            prev_sequence = matrix[:, col_index - 1]
            # The sequence being compared is the current column (C_i)
            current_sequence = matrix[:, col_index]

            # Mutation Step: Column 2 is Step 1, Column 3 is Step 2, etc.
            mutation_step = col_index - 1

            # 4. Iterate over all sites (rows)
            for row_index in 1:num_sites
                ref_aa = prev_sequence[row_index] # Reference is now the previous step's AA
                current_aa = current_sequence[row_index]

                # 5. Check for Mutation (change from the immediately preceding sequence)
                if current_aa != ref_aa
                    # Mutation found! Write to the file
                    # Format: Site   Ref_AA   New_AA   Mutation_Step
                    println(io, "$row_index\t$ref_aa\t$current_aa\t$mutation_step")
                end
            end
        end
    end

    if verbose == true
        @info "Step-wise mutation report successfully saved to: $filepath"
    end
    return filepath
end

 





N_steps = L;
N_copies = 1000;
fam_name = "DBD"
out_folder = "../$(fam_name)_traj_langevin/"

@time for idx in 1:size(clean_nat_msa,2)

    start_msa = hcat([clean_nat_msa[:,idx] for i in 1:N_copies]...); 
    res = run_evolution(start_msa, 
        h_dbd, 
        J_dbd,
        p = 0., 
        temp = 1., 
        N_steps = N_steps,  
        each_step = 1, 
        verbose = false);
    
    trajs = zeros(Int8, L, N_steps, N_copies);  
    for t in 1:N_steps
        trajs[:,t,:] .= res.step_msa[t]
    end

    for n in 1:N_copies
        output_file = joinpath(out_folder,"$(fam_name)_wt$(idx)_copy$(n).txt")
        find_mutations(trajs[:,:,n], output_file)
    end
    
    if idx % 100 == 0
        println("$(idx/M) % ...")
    end
end 

    
    
    
    
